<a href="https://colab.research.google.com/github/Suhyun77/project3/blob/main/project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Requirements**

## **반드시 구현되어야 하는 기능**

### **1. 데이터 가져오기(Pull)**

여러분이 수집하는 데이터를 정하는 것부터 시작합니다. 데이터는 다양한 방법으로 존재할 수 있습니다. 아래는 작업의 예시입니다. 데이터를 수집하는 작업을 진행해야합니다.

- 파일 다운로드 및 파일 읽기
- 데이터 스크레이핑 & 크롤링
- API 를 이용한 데이터 수집

Advanced

- 일정 시간 간격으로 데이터 수집(cron, 윈도우 스케줄링, APScheduler …)
- 동적 스크레이핑 (셀레니움 … )

---

### **2. 데이터 저장(Store)**

1.에서 가져온 데이터를 영원히 여러분의 메모리에 남길 수 없습니다. 가져온 데이터를 데이터베이스에 저장해야합니다.

- 관계형 데이터베이스 (sqlite, postgreSQL …)
- NoSQL 데이터베이스 (MongoDB … )

Advanced

- 로컬 데이터베이스를 배포해서 사용하는 방법

---

### **3. API 서비스 개발 (Service)**

수집된 데이터베이스의 데이터를 기반으로 Section 2 에서 배운 모델을 구성합니다. 먼저, 모델의 사용은 localhost API 로 개발해서 작업증명(Proof-of-Concept) 를 진행합니다.

- 개발한 모델이 API 로 사용가능해야 합니다.

Advanced

- API 를 다른 개발자들이 사용할 수 있도록 배포
- 서비스 사용자의 API 사용을 도와주기 위해, GUI 를 구성합니다. (부트스트랩 …)
- API 사용 결과가 데이터베이스에 저장될 수 있도록 변경

---

### **4. 데이터 분석용 대시보드 개발**

데이터베이스의 데이터를 기반으로 대시보드에 자신의 의견을 피력하기 위한 그래프를 구성합니다. 데이터 분석은 대시보드를 통해 진행되어야 하며, 만들어야할 그래프의 최소 개수는 3개 입니다.

- 데이터베이스의 데이터를 기반으로 EDA 를 진행합니다.
- 구글 데이터스튜디오와 metabase 는 기본 점수입니다.

Advanced

- `3.API 서비스 개발 (Service)` 의 모델 결과가 EDA 에 포함되도록 그래프를 구성합니다.
- Redash 를 이용한 대시보드 개발
- 대시보드 배포

# 서울대공원 동식물원 입장객 예측 AI봇 

# **EDA**

In [1]:
import pandas as pd
import datetime

In [2]:
### 2004년 
def make_df2(year, month): 
    excel = f'/content/서울대공원 입장객 정보_{year}년.xls'
    month = f'{month}월' 
    df = pd.read_excel(excel, sheet_name = month)
    df = pd.concat([df.iloc[5:-1, :2], df.iloc[5:-1, 7:15]], axis=1)
    df.columns = ['날짜','요일', '유료소계', '어른', '청소년',	'어린이', '외국인', '단체입장', '무료소계', '총계']
    df.reset_index(inplace=True, drop=True)
    globals()['df_{}_{}'.format(year, month)] = df
    return globals()['df_{}_{}'.format(year, month)]

# 해당 년도 데이터 합치기
df_2004 = pd.concat([make_df2(2004, 7), make_df2(2004, 8), make_df2(2004, 9), make_df2(2004, 10), make_df2(2004, 11), make_df2(2004, 12)])
df_2004['날짜'] = '2004년' + df_2004['날짜']
# print(len(df_2004))
# df_2004



### 2005년 ~ 2013년
for i in range(2005, 2014):
    globals()['df_{}'.format(i)] = pd.concat([make_df2(i, 1), make_df2(i, 2), make_df2(i, 3), make_df2(i, 4), make_df2(i, 5), make_df2(i, 6), make_df2(i, 7), make_df2(i, 8), make_df2(i, 9), make_df2(i, 10), make_df2(i, 11), make_df2(i, 12)])
    globals()['df_{}'.format(i)]['날짜'] = f'{i}년' + globals()['df_{}'.format(i)]['날짜']
    # print(f'{i}년 : ', len(globals()['df_{}'.format(i)]))
    df_2004 = pd.concat([df_2004, globals()['df_{}'.format(i)]])

# print(df_2004)

### 2014년
df_2014 = pd.concat([make_df2(2014, 1), make_df2(2014, 3), make_df2(2014, 4), make_df2(2014, 5), make_df2(2014, 6), make_df2(2014, 7), make_df2(2014, 8), make_df2(2014, 9), make_df2(2014, 10), make_df2(2014, 11), make_df2(2014, 12)])
df_2014['날짜'] = '2014년' + df_2014['날짜']

df = pd.concat([df_2004, df_2014])

# print(df)

### date
def datechange(date):
    x = datetime.datetime.strptime(date, "%Y년%m월%d일").date()
    return x

df['날짜'] = df['날짜'].apply(datechange)
df.reset_index(inplace=True, drop=True)
df.drop(['유료소계', '무료소계'], inplace=True, axis=1)
df

,날짜,요일,어른,청소년,어린이,외국인,단체입장,총계
0,2004-07-01,목,936,1,152,0,105,1695
1,2004-07-02,금,592,17,92,0,318,1240
2,2004-07-03,토,2653,50,379,0,734,3852
3,2004-07-04,일,390,0,53,0,106,694
4,2004-07-05,월,734,9,65,0,0,1545
...,...,...,...,...,...,...,...,...
3774,2014-12-27,토,1647,131,228,63,34,3572
3775,2014-12-28,일,1763,84,272,58,0,3506
3776,2014-12-29,월,664,45,164,66,0,1705
3777,2014-12-30,화,638,42,142,37,0,1663


In [3]:
#### 2015년 ~ 2020년 : 데이터 입력 형식 다름
def make_df2(year, month): 
    excel = f'/content/서울대공원 입장객 정보_{year}년.xls'
    month = f'{month}월' 
    df = pd.read_excel(excel, sheet_name = month)
    df = pd.concat([df.iloc[5:-1, :2], df.iloc[5:-1, 7:15]], axis=1)
    df.columns = ['날짜','요일', '유료소계', '어른', '청소년',	'어린이', '외국인', '단체입장', '무료소계', '총계']
    df.reset_index(inplace=True, drop=True)
    globals()['df_{}_{}'.format(year, month)] = df
    return globals()['df_{}_{}'.format(year, month)]



df_2015 = pd.concat([make_df2(2015, 1), make_df2(2015, 2), make_df2(2015, 3), make_df2(2015, 4), make_df2(2015, 5), make_df2(2015, 6), make_df2(2015, 7), make_df2(2015, 8), make_df2(2015, 9), make_df2(2015, 10), make_df2(2015, 11), make_df2(2015, 12)])
df_2016 = pd.concat([make_df2(2016, 1), make_df2(2016, 2), make_df2(2016, 3), make_df2(2016, 4), make_df2(2016, 5), make_df2(2016, 6), make_df2(2016, 7), make_df2(2016, 8), make_df2(2016, 9), make_df2(2016, 10), make_df2(2016, 11), make_df2(2016, 12)])
df_2017 = pd.concat([make_df2(2017, 3), make_df2(2017, 4), make_df2(2017, 5), make_df2(2017, 6), make_df2(2017, 7), make_df2(2017, 8), make_df2(2017, 9), make_df2(2017, 10), make_df2(2017, 11), make_df2(2017, 12)])
df_2018 = pd.concat([make_df2(2018, 1), make_df2(2018, 2), make_df2(2018, 3), make_df2(2018, 4), make_df2(2018, 5), make_df2(2018, 6), make_df2(2018, 7), make_df2(2018, 8), make_df2(2018, 9), make_df2(2018, 10), make_df2(2018, 11), make_df2(2018, 12)])
df_2019 = pd.concat([make_df2(2019, 1), make_df2(2019, 2), make_df2(2019, 3), make_df2(2019, 4), make_df2(2019, 5), make_df2(2019, 6), make_df2(2019, 7), make_df2(2019, 8), make_df2(2019, 9), make_df2(2019, 10), make_df2(2019, 11), make_df2(2019, 12)])
df_2020 = pd.concat([make_df2(2020, 1), make_df2(2020, 2), make_df2(2020, 3), make_df2(2020, 4)])

### date
df2 = pd.concat([df_2015, df_2016, df_2017, df_2018, df_2019, df_2020])
df2['날짜'] = pd.to_datetime(df2['날짜'])

df2

,날짜,요일,유료소계,어른,청소년,어린이,외국인,단체입장,무료소계,총계
0,2015-01-03,토,1996,1634,104,258,45,60,1318,3314
1,2015-01-04,일,3312,2763,144,405,28,30,1889,5201
2,2015-01-05,월,707,565,32,110,45,0,871,1578
3,2015-01-06,화,369,244,47,78,17,50,886,1255
4,2015-01-07,수,288,213,52,23,14,0,718,1006
...,...,...,...,...,...,...,...,...,...,...
23,2020-04-25,토,5425,4659,109,657,209,39,1874,7299
24,2020-04-26,일,9406,8128,185,1093,209,0,1774,11180
25,2020-04-27,월,1359,1227,23,109,209,0,745,2104
26,2020-04-28,화,1766,1564,19,183,209,0,943,2709


In [4]:
#### final_df
final_df = pd.concat([df, df2])
final_df.drop(['유료소계', '무료소계'], axis=1, inplace=True)
final_df.reset_index(inplace=True, drop=True)


### 요일 > 주말 여부 (yes:1, no:0)
for i in range(len(final_df)):
    if final_df['요일'][i] == '토':
        final_df['요일'][i] = 1
    elif final_df['요일'][i] == '일':
        final_df['요일'][i] = 1
    else:
        final_df['요일'][i] = 0
final_df.rename(columns = {'요일':'주말 여부'}, inplace=True)


### 공휴일 여부 (yes:1, no:0)
holiday = []



final_df['날짜'] = pd.to_datetime(final_df['날짜'])
final_df.set_index('날짜', inplace=True)
final_df = final_df.astype(int)
final_df.reset_index(inplace=True)
final_df

,날짜,주말 여부,어른,청소년,어린이,외국인,단체입장,총계
0,2004-07-01,0,936,1,152,0,105,1695
1,2004-07-02,0,592,17,92,0,318,1240
2,2004-07-03,1,2653,50,379,0,734,3852
3,2004-07-04,1,390,0,53,0,106,694
4,2004-07-05,0,734,9,65,0,0,1545
...,...,...,...,...,...,...,...,...
5491,2020-04-25,1,4659,109,657,209,39,7299
5492,2020-04-26,1,8128,185,1093,209,0,11180
5493,2020-04-27,0,1227,23,109,209,0,2104
5494,2020-04-28,0,1564,19,183,209,0,2709


In [5]:
### merge 기온 to final_df
temperature = pd.read_excel('/content/temperature.xlsx', engine='openpyxl')
final_df = pd.merge(final_df, temperature, left_on='날짜', right_on='날짜', how='left')
final_df

,날짜,주말 여부,어른,청소년,어린이,외국인,단체입장,총계,평균기온(℃),최저기온(℃),최고기온(℃)
0,2004-07-01,0,936,1,152,0,105,1695,24.0,22.0,26.7
1,2004-07-02,0,592,17,92,0,318,1240,23.1,21.3,24.2
2,2004-07-03,1,2653,50,379,0,734,3852,24.4,22.0,26.8
3,2004-07-04,1,390,0,53,0,106,694,21.1,19.7,22.7
4,2004-07-05,0,734,9,65,0,0,1545,22.8,19.5,26.2
...,...,...,...,...,...,...,...,...,...,...,...
5491,2020-04-25,1,4659,109,657,209,39,7299,13.1,9.9,17.1
5492,2020-04-26,1,8128,185,1093,209,0,11180,12.3,6.8,19.0
5493,2020-04-27,0,1227,23,109,209,0,2104,12.9,8.7,18.3
5494,2020-04-28,0,1564,19,183,209,0,2709,13.9,7.8,19.9


In [6]:
### 강수량 feature null값 채우기
rainfall = pd.read_excel('/content/rainfall.xlsx', engine='openpyxl')
rainfall.fillna(value=0, inplace=True)
# rainfall.isnull().sum()

### merge 강수량 to final_df
final_df = pd.merge(final_df, rainfall, left_on='날짜', right_on='날짜', how='left')
final_df

,날짜,주말 여부,어른,청소년,어린이,외국인,단체입장,총계,평균기온(℃),최저기온(℃),최고기온(℃),강수량(mm)
0,2004-07-01,0,936,1,152,0,105,1695,24.0,22.0,26.7,4.5
1,2004-07-02,0,592,17,92,0,318,1240,23.1,21.3,24.2,10.5
2,2004-07-03,1,2653,50,379,0,734,3852,24.4,22.0,26.8,10.5
3,2004-07-04,1,390,0,53,0,106,694,21.1,19.7,22.7,62.0
4,2004-07-05,0,734,9,65,0,0,1545,22.8,19.5,26.2,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5491,2020-04-25,1,4659,109,657,209,39,7299,13.1,9.9,17.1,0.0
5492,2020-04-26,1,8128,185,1093,209,0,11180,12.3,6.8,19.0,0.0
5493,2020-04-27,0,1227,23,109,209,0,2104,12.9,8.7,18.3,0.0
5494,2020-04-28,0,1564,19,183,209,0,2709,13.9,7.8,19.9,0.0


In [7]:
# 날짜 : datetime -> string으로 변환해 DB 저장
final_df2 = final_df.copy() 
final_df2['날짜']=final_df2['날짜'].astype(str)
final_df2.info()
final_df2.to_excel('/content/Childpark_data_DB.xlsx')
final_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5496 entries, 0 to 5495
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   날짜       5496 non-null   object 
 1   주말 여부    5496 non-null   int64  
 2   어른       5496 non-null   int64  
 3   청소년      5496 non-null   int64  
 4   어린이      5496 non-null   int64  
 5   외국인      5496 non-null   int64  
 6   단체입장     5496 non-null   int64  
 7   총계       5496 non-null   int64  
 8   평균기온(℃)  5496 non-null   float64
 9   최저기온(℃)  5496 non-null   float64
 10  최고기온(℃)  5495 non-null   float64
 11  강수량(mm)  5496 non-null   float64
dtypes: float64(4), int64(7), object(1)
memory usage: 558.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5496 entries, 0 to 5495
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   날짜       5496 non-null   object 
 1   주말 여부    5496 non-null   int64  
 2   어른       5496 non-null   int64  


In [8]:
# 2014 데이터 read
# def read(year, firstmonth, lastmonth):
#     for i in range(firstmonth, lastmonth + 1):
#         excel = f'/content/서울대공원 입장객 정보_{year}년.xls'
#         sheetname = f'{i}월'

#         df = pd.read_excel(excel, sheet_name=sheetname)
#         print(df)
#         print(df.iloc[:, :7])
#         df = pd.concat([df_2014_7.iloc[5:-1, 0], df_2014_7.iloc[5:-1, 7:15]], axis=1)
#         print(df_2014_7.iloc[5:-1, 0])
#         print(df_2014_7.iloc[5:-1, 7:15])
#         df.columns = ['날짜', '유료소계', '어른', '청소년',	'어린이', '외국인', '단체입장', '무료소계', '총계']
#         df.reset_index(inplace=True, drop=True)
        
#         # 파이썬 동적 변수
#         globals()['df_{}_{}'.format(year, i)] = df
#         print(globals()['df_{}_{}'.format(year, i)])  # 생성된 df 확인

# 데이터 읽기
# def make_df(year, firstmonth, lastmonth):
#     for i in range(firstmonth, lastmonth + 1):
#         excel = f'/content/서울대공원 입장객 정보_{year}년.xls'
#         month = f'{i}월'
        
#         # 필요한 데이터만 남기기
#         df = pd.read_excel(excel, sheet_name=month)
#         df = pd.concat([df.iloc[5:-1, 0], df.iloc[5:-1, 7:15]], axis=1)
#         df.columns = ['날짜', '유료소계', '어른', '청소년',	'어린이', '외국인', '단체입장', '무료소계', '총계']
#         df.reset_index(inplace=True, drop=True)
#         globals()['df_{}_{}'.format(year, i)] = df
#         return globals()['df_{}_{}'.format(year, i)]

# **ML Model**

In [10]:
final_df

,날짜,주말 여부,어른,청소년,어린이,외국인,단체입장,총계,평균기온(℃),최저기온(℃),최고기온(℃),강수량(mm)
0,2004-07-01,0,936,1,152,0,105,1695,24.0,22.0,26.7,4.5
1,2004-07-02,0,592,17,92,0,318,1240,23.1,21.3,24.2,10.5
2,2004-07-03,1,2653,50,379,0,734,3852,24.4,22.0,26.8,10.5
3,2004-07-04,1,390,0,53,0,106,694,21.1,19.7,22.7,62.0
4,2004-07-05,0,734,9,65,0,0,1545,22.8,19.5,26.2,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5491,2020-04-25,1,4659,109,657,209,39,7299,13.1,9.9,17.1,0.0
5492,2020-04-26,1,8128,185,1093,209,0,11180,12.3,6.8,19.0,0.0
5493,2020-04-27,0,1227,23,109,209,0,2104,12.9,8.7,18.3,0.0
5494,2020-04-28,0,1564,19,183,209,0,2709,13.9,7.8,19.9,0.0


In [9]:
target